In [52]:
import numpy as np
import random
import matplotlib.pyplot as plt
import keras
from keras.models import Model
from keras.layers import *
from keras.activations import *
from keras.datasets import mnist

In [53]:
img_rows, img_cols = 28, 28

# the data, shuffled and split between train and test sets
(X_train, y_train), (X_test, y_test) = mnist.load_data()

X_train = X_train.reshape(X_train.shape[0], 1, img_rows, img_cols)
X_test = X_test.reshape(X_test.shape[0], 1, img_rows, img_cols)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

In [54]:
def gen_loss_function(input):
    loss = log(discriminator(Generator(input)))
    return loss

def discriminator_loss_function(input):
    loss = log(discriminator(input))
    return loss

def make_trainable(net, val):
    net.trainable = val
    for l in net.layers:
        l.trainable = val

In [55]:
# Generator Model
g_input = Input(shape=(1,))
H = Activation(ReLU)(g_input)
H = Dense(1024)(H)
H = Activation(ReLU)(H)
H = Dense(768)(H)
generator = Model(g_input,H)
generator.compile(loss='binary_crossentropy', optimizer=opt)
generator.summary()

# Discriminator Model
d_input = Input(shape=(28,28))
H = Flatten()(d_input)
H = Activation(ReLU)(H)
H = Dense()(H)
H = Dropout(dropout_rate)(H)
d_V = Dense(2,activation='softmax')(H)
discriminator = Model(d_input,d_V)
discriminator.compile(loss='categorical_crossentropy', optimizer=dopt)
discriminator.summary()

ValueError: Layer dense_21 was called with an input that isn't a symbolic tensor. Received type: <class 'keras.layers.advanced_activations.ReLU'>. Full input: [<keras.layers.advanced_activations.ReLU object at 0x7f46bb4aa710>]. All inputs to the layer should be tensors.

In [ ]:
def noise(shape):
    X = np.random.normal(0,1,shape)
    return X

In [ ]:
epochs =10000

In [ ]:
# Initial training for Discriminator
ntrain = 10000
trainidx = random.sample(range(0,X_train.shape[0]), ntrain)
XT = X_train[trainidx,:,:,:]

generator = Generator()
discriminator = discriminator()
noise_gen = np.random.uniform(0,1,size=[XT.shape[0],100])
generated_images = generator.predict(noise_gen)
X = np.concatenate((XT, generated_images))
n = XT.shape[0]
y = np.zeros([2*n,2])
y[:n,1] = 1
y[n:,0] = 1

make_trainable(discriminator,True)
discriminator.fit(X,y, nb_epoch=1, batch_size=128)
y_hat = discriminator.predict(X)

In [ ]:
for i in range(epochs):
    